# Network Operations
## Pre-Processing

In [2]:
# nuclio: ignore
import nuclio

Define the MLRun environment

In [3]:
from mlrun import new_function, code_to_function, get_run_db, mount_v3io, NewTask, mlconf, new_model_server, run_local
mlconf.dbpath = 'http://mlrun-api:8080'

In [ ]:
%nuclio config kind = "job"

## Function

In [4]:
# nuclio: start-code

In [2]:
import os
import pandas as pd

In [11]:
def aggregate(context,
              df_artifact, 
              keys=None, 
              metrics=None, 
              labels=None, 
              metric_aggs=['mean'], 
              label_aggs=['max'], 
              suffix=None, 
              window=3, 
              center=False, 
              inplace=False):
    
    context.logger.info(df_artifact)
    input_df = pd.read_parquet(df_artifact)
    
    # Verify there is work to be done
    if not (metrics or labels):
        context.log_artifact('df', input_df)
        return input_df
    
    # Select the correct indexes
    if keys:
        current_index = input_df.index.names
        indexes_to_drop = [col for col in input_df.index.names if col not in keys]
        df = input_df.reset_index(level=indexes_to_drop)
    else:
        df = input_df
    
    # For each metrics
    if metrics:
        metrics_df = df.loc[:, metrics].rolling(window=window,
                                                center=center).aggregate(metric_aggs)
        
        # Flatten all the aggs
        metrics_df.columns = ['_'.join(col).strip() for col in metrics_df.columns.values]
        
        # Add suffix
        if suffix:
            metrics_df.columns = [f'{metric}_{suffix}' for metric in metrics_df.columns]
            
        if append_to_df:
            final_df = pd.merge(input_df, metrics_df, suffixes=('', suffix), left_index=True, right_index=True)
        else:
            final_df = metrics_df

    # For each label
    if labels:
        labels_df = df.loc[:, labels].rolling(window=window,
                                              center=center).aggregate(label_aggs)
        # Flatten all the aggs
        labels_df.columns = ['_'.join(col).strip() for col in labels_df.columns.values]
        
        # Add suffix
        if suffix:
            labels_df.columns = [f'{label}_{suffix}' for label in labels_df.columns]
            
        if metrics:
            final_df = pd.merge(final_df, labels_df, suffixes=('', suffix), left_index=True, right_index=True)   
        else:
            if append_to_df:
                final_df = pd.merge(input_df, labels_df, suffixes=('', suffix), left_index=True, right_index=True)      
            else:
                final_df = labels_df
        
    # Save the result dataframe
    # TODO: Change to log_datset
    context.log_dataset(key='aggregate', 
                        df=final_df, 
                        format='parquet')

In [7]:
# nuclio: end-code

## Test
Define client to get metrics sample

In [9]:
# Define V3IO Client
import v3io_frames as v3f
client = v3f.Client('framesd:8081', container='bigdata')

# Define base dirs
project_dir = os.path.join('/', 'User', 'demo-network-operations')

In [9]:
metrics = client.read('tsdb', 'netops_metrics', multi_index=True)
metrics_pq = os.path.join(project_dir, 'data', 'metrics.pq')
metrics.to_parquet(metrics_pq, engine='pyarrow', index=True)
metrics.head(2)

,,,,cpu_utilization,cpu_utilization_is_error,is_error,latency,latency_is_error,packet_loss,packet_loss_is_error,throughput,throughput_is_error
time,company,data_center,device,,,,,,,,,
2020-03-10 08:52:37.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,78.396905,0.0,0.0,0.0,0.0,0.000000,0.0,251.553131,0.0
2020-03-10 08:52:42.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,73.343463,0.0,0.0,0.0,0.0,0.113283,0.0,247.769434,0.0


### Local Test
Define the aggregate test task

In [14]:
aggregate_task = NewTask(name='aggregate',
                         project='network-operations',
                         params={'df_artifact': os.path.join(project_dir, 'data', 'metrics.pq'),
                                 'metrics': ['cpu_utilization'],
                                 'labels': ['is_error'],
                                 'metric_aggs': ['mean', 'sum'],
                                 'label_aggs': ['max'],
                                 'suffix': 'daily',
                                 'inplace': False,
                                 'window': 5,
                                 'center': True},
                         handler=aggregate)

rolling(window, ....)
aggregate([])
{
    'aggregate': [<cols>],
}

Join([.....], )
{
    
}

Operators_fun(
-- Load data
-- Functions
 [
     {'aggregate': {'cols': 'agg'}},
     {'save_dataset': {'to': '....'}},
     {'join': [previous_step.outputs['data'], static_data_1]},
     {'aggregate': {'cols': 'agg'}},
 ]
)

In [15]:
run_local(aggregate_task)

[mlrun] 2020-03-12 15:31:27,423 artifact path is not defined or is local,artifacts will not be visible in the UI
[mlrun] 2020-03-12 15:31:27,431 warning!, server (0.4.5) and client (0.4.6) ver dont match
[mlrun] 2020-03-12 15:31:27,431 starting run aggregate uid=d0e64ae914ae480aaf22cf9665b185e3  -> http://mlrun-api:8080
[mlrun] 2020-03-12 15:31:27,448 warning!, server (0.4.5) and client (0.4.6) ver dont match
[mlrun] 2020-03-12 15:31:27,458 /User/demo-network-operations/data/metrics.pq
[mlrun] 2020-03-12 15:31:27,724 log artifact aggregate at /User/demo-network-operations/data/aggregate.pq, size: 37174, db: Y



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...b185e3,0,Mar 12 15:31:27,completed,aggregate,kind=handlerowner=adminhost=jupyter-78ddb8b99c-rpmd7,,"df_artifact=/User/demo-network-operations/data/metrics.pqmetrics=['cpu_utilization']labels=['is_error']metric_aggs=['mean', 'sum']label_aggs=['max']suffix=dailyappend_to_df=Truewindow=5center=Truesave_to=/User/demo-network-operations/data/aggregate.pq",,aggregate


to track results use .show() or .logs() or in CLI: 
!mlrun get run d0e64ae914ae480aaf22cf9665b185e3 --project network-operations , !mlrun logs d0e64ae914ae480aaf22cf9665b185e3 --project network-operations
[mlrun] 2020-03-12 15:31:27,779 run executed, status=completed


### Test on cluster

Convert the code to an MLRun function

In [10]:
fn = code_to_function('aggregate', 
                      code_output=os.path.join(project_dir, 'src', 'aggregate.py'),
                      kind='job').apply(mount_v3io())
fn.export(os.path.join(project_dir, 'yaml', 'aggregate.yaml'))

[mlrun] 2020-03-12 15:20:52,376 function spec saved to path: /User/demo-network-operations/yaml/aggregate.yaml


In [16]:
fn.deploy()

[mlrun] 2020-03-12 15:31:39,485 running build to add mlrun package, set with_mlrun=False to skip if its already in the image
[mlrun] 2020-03-12 15:31:39,486 starting remote build, image: .mlrun/func-default-aggregate-latest


True

In [17]:
fn.run(aggregate_task)

[mlrun] 2020-03-12 15:31:40,931 artifact path is not defined or is local,artifacts will not be visible in the UI
[mlrun] 2020-03-12 15:31:40,931 starting run aggregate uid=94d7854773ee430fb65b47764d4772fb  -> http://mlrun-api:8080
[mlrun] 2020-03-12 15:31:40,987 Job is running in the background, pod: aggregate-qbft8
[mlrun] 2020-03-12 15:31:45,693 server (0.4.5) and client (0.4.4) ver dont match
[mlrun] 2020-03-12 15:31:45,736 server (0.4.5) and client (0.4.4) ver dont match
[mlrun] 2020-03-12 15:31:45,745 /User/demo-network-operations/data/metrics.pq
[mlrun] 2020-03-12 15:31:45,876 log artifact aggregate at /User/demo-network-operations/data/aggregate.pq, size: 37175, db: Y

[mlrun] 2020-03-12 15:31:45,885 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...4772fb,0,Mar 12 15:31:45,completed,aggregate,host=aggregate-qbft8kind=jobowner=admin,,"append_to_df=Truecenter=Truedf_artifact=/User/demo-network-operations/data/metrics.pqlabel_aggs=['max']labels=['is_error']metric_aggs=['mean', 'sum']metrics=['cpu_utilization']save_to=/User/demo-network-operations/data/aggregate.pqsuffix=dailywindow=5",,aggregate


to track results use .show() or .logs() or in CLI: 
!mlrun get run 94d7854773ee430fb65b47764d4772fb --project network-operations , !mlrun logs 94d7854773ee430fb65b47764d4772fb --project network-operations
[mlrun] 2020-03-12 15:31:47,092 run executed, status=completed


In [18]:
print(fn.to_yaml())

kind: job
metadata:
  name: aggregate
  tag: ''
  hash: 142b81f0b4140c847debc791f62b6b6e9f40f328
  project: ''
  categories: []
spec:
  command: /User/demo-network-operations/src/aggregate.py
  args: []
  image: .mlrun/func-default-aggregate-latest
  volumes:
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: 48bfe2e0-9870-4d11-9924-b20502a85363
        container: users
        subPath: /admin
    name: v3io
  volume_mounts:
  - mountPath: /User
    name: v3io
  env:
  - name: V3IO_API
    value: v3io-webapi.default-tenant.svc:8081
  - name: V3IO_USERNAME
    value: admin
  - name: V3IO_ACCESS_KEY
    value: 48bfe2e0-9870-4d11-9924-b20502a85363
  description: ''
  build:
    image: .mlrun/func-default-aggregate-latest
    commands: []
    code_origin: http://github.com/zilbermanor/demo-network-operations#22e21274be22a767435c8f3dd21169d8912e99cc:aggregate.ipynb
status:
  state: ready
  build_pod: ''



### Show results

In [12]:
df = pd.read_parquet('../data/metrics.pq')

In [15]:
df.rolling(window=5, center=True).sum()

,,,,cpu_utilization,cpu_utilization_is_error,is_error,latency,latency_is_error,packet_loss,packet_loss_is_error,throughput,throughput_is_error
time,company,data_center,device,,,,,,,,,
2020-03-10 08:52:37.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-10 08:52:42.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-10 08:52:47.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,355.621234,0.0,0.0,12.542095,0.0,2.573906,0.0,1283.408906,0.0
2020-03-10 08:52:52.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,360.114513,0.0,0.0,16.670036,0.0,2.710323,0.0,1277.847154,0.0
2020-03-10 08:52:57.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,384.055156,0.0,0.0,16.670036,0.0,2.597040,0.0,1270.497754,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-10 08:57:57.874000+00:00,Elliott_Inc,Baldwin_Views,1456313469721,335.951869,0.0,0.0,21.039117,0.0,2.213513,0.0,1144.925182,0.0
2020-03-10 08:58:02.874000+00:00,Elliott_Inc,Baldwin_Views,1456313469721,342.293920,0.0,0.0,23.681274,0.0,3.610201,0.0,1159.025662,0.0
2020-03-10 08:58:07.874000+00:00,Elliott_Inc,Baldwin_Views,1456313469721,337.549788,0.0,0.0,18.354520,0.0,2.057471,0.0,1208.098993,0.0


In [3]:
pd.read_parquet(os.path.join('/User/demo-network-operations', 'data', 'aggregate.pq'))

,,,,cpu_utilization,cpu_utilization_is_error,is_error,latency,latency_is_error,packet_loss,packet_loss_is_error,throughput,throughput_is_error,cpu_utilization_mean_daily,cpu_utilization_sum_daily,is_error_max_daily
time,company,data_center,device,,,,,,,,,,,,
2020-03-10 08:52:37.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,78.396905,0.0,0.0,0.000000,0.0,0.000000,0.0,251.553131,0.0,NaN,NaN,NaN
2020-03-10 08:52:42.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,73.343463,0.0,0.0,0.000000,0.0,0.113283,0.0,247.769434,0.0,NaN,NaN,NaN
2020-03-10 08:52:47.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,59.492871,0.0,0.0,0.000000,0.0,2.300774,0.0,278.961957,0.0,71.124247,355.621234,0.0
2020-03-10 08:52:52.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,73.483893,0.0,0.0,3.051355,0.0,0.000000,0.0,240.857938,0.0,72.022903,360.114513,0.0
2020-03-10 08:52:57.874000+00:00,Lambert__Watson_and_Stone,Cabrera_Ranch,6767536359526,70.904103,0.0,0.0,9.490740,0.0,0.159850,0.0,264.266446,0.0,76.811031,384.055156,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-10 08:57:57.874000+00:00,Elliott_Inc,Baldwin_Views,1456313469721,66.094782,0.0,0.0,3.251496,0.0,0.660784,0.0,238.154349,0.0,67.190374,335.951869,0.0
2020-03-10 08:58:02.874000+00:00,Elliott_Inc,Baldwin_Views,1456313469721,60.035049,0.0,0.0,5.759561,0.0,0.000000,0.0,244.279546,0.0,68.458784,342.293920,0.0
2020-03-10 08:58:07.874000+00:00,Elliott_Inc,Baldwin_Views,1456313469721,68.830267,0.0,0.0,4.006027,0.0,0.000000,0.0,226.150328,0.0,67.509958,337.549788,0.0
